#### Load remote demo NYC Yellow Taxi data to Dataframe

In [2]:
#Azure storage access info

blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = r""

#Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token)

print('Remote blob path: ' + wasbs_path)

# SPARK read parquet, note that it won't load any data yet by now
df = spark.read.parquet(wasbs_path)

#display(spark.sql('SELECT * FROM source LIMIT 10'))
df.count()

StatementMeta(, 707faa41-a074-4c1a-85e3-119a6f5c3183, 4, Finished, Available)

Remote blob path: wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow


1571671152

#### Save Dataframe to Lakehouse Files Area

In [3]:
df.write.mode("overwrite").format("parquet").partitionBy("puYear").save("Files/" + "raw_nyctlcYellow")

StatementMeta(, 707faa41-a074-4c1a-85e3-119a6f5c3183, 5, Finished, Available)

#### Load from Lakehouse Files area to Bronze Table then run Optimize to compact Files

In [2]:
df = spark.read.parquet("Files/raw_nyctlcYellow")
display(df)

StatementMeta(, 5d88a51b-c7b9-451f-aeb5-99491049620d, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, 49c92652-3a3f-40b7-99e5-363c3dc0632b)

In [3]:
df.write.mode("overwrite").format("delta").save("Tables/" + "bronze_nyctlcYellow")

StatementMeta(, 5d88a51b-c7b9-451f-aeb5-99491049620d, 5, Finished, Available)

In [1]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "Tables/" + "bronze_nyctlcYellow")

deltaTable.optimize().executeCompaction()

StatementMeta(, bfd6dc03-3d54-47f4-99b4-d6edebefd8ca, 3, Finished, Available)

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de